In [484]:
from pandas.tseries.offsets import MonthEnd
import warnings

import pandas as pd
import numpy as np

def es_dia_semana(fecha):
    return fecha.weekday() < 5  # Lunes=0, ..., Viernes=4 -> True, Sábado=5, Domingo=6 -> False
warnings.filterwarnings("ignore")
def calcular_promedios_semanales(row, df):
    fecha_referencia = row['fecha']
    fecha_inicio_previas = fecha_referencia - pd.DateOffset(weeks=3)
    fecha_fin_previas = fecha_referencia
    fecha_inicio_posteriores = fecha_referencia 
    fecha_fin_posteriores = fecha_referencia + pd.DateOffset(weeks=3)

    df_filtrado_previas = df[(df['fecha'] >= fecha_inicio_previas) & (df['fecha'] < fecha_fin_previas) & (df["cambiar"] == False)]
    df_filtrado_posteriores = df[(df['fecha'] > fecha_inicio_posteriores) & (df['fecha'] <= fecha_fin_posteriores) & (df["cambiar"] == False)]
    
    promedio_previas = df_filtrado_previas['casos'].mean() if not df_filtrado_previas.empty else np.nan
    promedio_posteriores = df_filtrado_posteriores['casos'].mean() if not df_filtrado_posteriores.empty else np.nan
    
    return promedio_previas, promedio_posteriores

        
df_base2=pd.read_csv("Bases//Base_Limpia_Added.csv")


# for y in range(2009,2017):
#     print(y)
#     df_0101=df_base2[(df_base2["fecha"]==str(y)+"-01-01")|(df_base2["fecha"]==str(y)+"-01-02")]

#     n=(len(df_0101)//2)
#     random_indices = np.random.choice(df_0101.index, n, replace=False)

#     df_base2.loc[random_indices,"fecha"]=str(y-1)+"-12-31"

    

df_final2=pd.DataFrame()
festivos_argentina = [
    #'01-01',  # Año Nuevo NO CONTAR PRIMERO
    #'02-12',  # Carnaval (primer día)
    #'02-13',  # Carnaval (segundo día)
    '03-24',  # Día de la Memoria por la Verdad y la Justicia
    #'03-29',  # Viernes Santo
    #'05-01',  # Día del Trabajador NO CONTAR PRIMERO
    '05-25',  # Día de la Revolución de Mayo
    #'06-17',  # Día Paso a la Inmortalidad del General Martín Miguel de Güemes
    '06-20',  # Día Paso a la Inmortalidad del General Manuel Belgrano
    '07-09',  # Día de la Independencia
    # '08-19',  # Día Paso a la Inmortalidad del General José de San Martín

    # '10-12',  # Día del Respeto a la Diversidad Cultural (antes conocido como Día de la Raza)

    '12-08',  # Día del Respeto a la Diversidad Cultural (antes conocido como Día de la Raza)
    '12-24',  # Noche buena
    '12-25',  # Navidad
    '12-31',  # Ultimo Día
]
def crear_df_aux(fecha_ym):

    fecha_inicio = pd.to_datetime(fecha_ym, format='%Y-%m')   
    fecha_fin = fecha_inicio + MonthEnd(1)    
    fechas = pd.date_range(start=fecha_inicio, end=fecha_fin)
    df_aux = pd.DataFrame({'fecha': fechas})
    
    return df_aux



# # for p in ["Buenos Aires"]:
# for p in df_base2["provincia"].unique():

for p in ["Salta","Jujuy","CABA","Santa Fe"]:
    print(p)
    df=df_base2.loc[df_base2["provincia"]==p]
    df=pd.DataFrame(pd.pivot_table(df,index=['fecha'],values=["id"],aggfunc="count").to_records())
    df.columns=["fecha","casos"]



    df['fecha'] = pd.to_datetime(df['fecha'])

    df['Dia semana'] = df['fecha'].apply(es_dia_semana)


    df["Dia Festivo"]=0

    df.loc[df["fecha"].astype(str).str[-5:].isin(festivos_argentina),"Dia Festivo"]=1
    df['cambiar'] = (df['fecha'].dt.day == 1) | (df['fecha'].dt.day == 15) | (df['Dia Festivo']== 1) 

    aux=df.apply(calcular_promedios_semanales, axis=1, args=(df,))
    l_aux=[]
    for s in aux:
        l_aux.append(tuple(s))
    df_aux=pd.DataFrame(l_aux,columns=['prev_', 'post_'])

    df2=pd.concat([df,df_aux],axis=1)
    df2["prom_cases"]=df2[["prev_", "post_"]].mean(axis=1, skipna=True)

    df2["casos corregidos"]=df["casos"]
    df2.loc[(df2["cambiar"]==True) | (df['Dia semana']==False) ,"casos corregidos"]=df2.loc[(df2["cambiar"]==True)| (df['Dia semana']==False),"prom_cases"]

    
    
    df2["Mes_Año"]=df2["fecha"].astype(str).str[:7]

    df2["casos_corr_2"]=""
    df_final=pd.DataFrame()
    for m in df2["Mes_Año"].unique():

        df3=df2[df2["Mes_Año"]==m].reset_index(drop=True)
        
        df_aux = crear_df_aux(m)
        df3=pd.merge(df_aux,df3,on="fecha",how="left").fillna(0)
        
        df3.loc[df3["casos corregidos"]<df3["casos corregidos"].mean()*0.4,"casos corregidos"]=df3["casos corregidos"].mean()
        df3.loc[df3["casos corregidos"]>df3["casos corregidos"].mean()*1.6,"casos corregidos"]=df3["casos corregidos"].mean()

        df3["casos corregidos"]=df3["casos corregidos"].round(0)
        # if m[-2:]=="12":

        #     df3["casos"]=df3["casos"]+0.075
        df3["exceso"]=df3["casos"]-df3["casos corregidos"]
        exceso=int(df3["exceso"].sum())
        exceso_dia=int(df3["exceso"].sum()/len(df3))

        df3["casos_corr_2"]=df3["casos corregidos"]+exceso_dia
        np.random.seed(42)

        n=int(df3["casos"].sum()-df3["casos_corr_2"].sum())
        while n>len(df_aux):
            df3["casos_corr_2"]=df3["casos_corr_2"]+1
            n=n-len(df_aux)

        while n<-len(df_aux):
            df3["casos_corr_2"]=df3["casos_corr_2"]-1      
            n=n+len(df_aux)  
        if n<0:

            try:
                random_indices = np.random.choice(df3.index, -n, replace=False)
                selected_rows = df3.loc[random_indices]
                selected_rows['casos_corr_2'] -= 1
            except:
                display(df3)
                print(p,n)
        else:      

            random_indices = np.random.choice(df3.index, n, replace=False)
            selected_rows = df3.loc[random_indices]


            selected_rows['casos_corr_2'] += 1
        df3=pd.concat([selected_rows,df3]).drop_duplicates(subset="fecha",keep="first").sort_values(by="fecha")

        df_final=pd.concat([df_final,df3])
            
        df_final=df_final.drop(["prev_","post_","prom_cases","casos corregidos","exceso","Mes_Año"],axis=1)
    df_final["provincia"]=p
    
    df_final2=pd.concat([df_final2,df_final])

# df_final2.loc[df_final2['fecha'].dt.month==12,"casos_corr_2"]=df_final2.loc[df_final2['fecha'].dt.month==12,"casos_corr_2"]*1.04


Salta
Jujuy
CABA
Santa Fe


In [485]:
df_final3=df_final2.copy()




In [486]:

df_fial=df_final3.copy()

pd.pivot_table(df_fial,index=df_fial["fecha"].dt.month,values="casos_corr_2",aggfunc="sum")

df_fial.groupby([df_fial["fecha"].dt.month])[["casos_corr_2"]].sum()

,casos_corr_2
fecha,
1,3646.0
2,2814.0
3,3257.0
4,3263.0
5,3120.0
6,3034.0
7,3041.0
8,3373.0
9,3491.0


In [487]:

df_final["casos"].sum(),df_final["casos_corr_2"].sum()
df_final2.to_csv("Bases//Base_Casos_Act.csv",index=False)




In [488]:
1/0

ZeroDivisionError: division by zero

In [489]:
df_aux=df_final2[df_final2["provincia"].isin(["CABA"])].groupby("fecha")["casos_corr_2"].sum()
df_aux



fecha
2009-01-01    2.0
2009-01-02    3.0
2009-01-03    3.0
2009-01-04    3.0
2009-01-05    2.0
             ... 
2022-12-27    4.0
2022-12-28    4.0
2022-12-29    5.0
2022-12-30    4.0
2022-12-31    3.0
Name: casos_corr_2, Length: 5113, dtype: float64

In [ ]:
df_base2.loc[(df_base2["fecha"].str[-5:]=="01-02")&df_base2["provincia"].isin(["Salta","Jujuy"])]

,id,fnyear,sexo,edad,provincia,id_provincia,departamento,id_departamento,localidad,fecha,...,% de población sin obra social ni prepaga,DD,FF,PNM,TEMP,HUM,Dia Festivo,1 o 15,dia_semana,es_dia_laboral
21637,75190,2011,F,71.0,Salta,66,CAPITAL,28,SALTA,2011-01-02,...,41.91,196.561359,10.649395,1252.128453,30.400543,89.218105,0,0,6,0
36762,92824,2013,F,19.0,Jujuy,38,EL CARMEN,14,PERICO,2013-01-02,...,51.86,184.707990,7.662346,1950.963492,22.607795,76.680901,0,0,2,1
37305,93524,2013,F,25.0,Jujuy,38,GRAL MANUEL BELGRANO,21,SAN SALVADOR DE JUJUY,2013-01-02,...,44.96,184.707990,7.662346,1950.963492,22.607795,76.680901,0,0,2,1
38041,94448,2013,F,35.0,Jujuy,38,TILCARA,94,MAIMARA,2013-01-02,...,61.00,184.707990,7.662346,1950.963492,22.607795,76.680901,0,0,2,1
40405,97366,2013,M,21.0,Jujuy,38,SAN PEDRO,63,SAN PEDRO DE JUJUY,2013-01-02,...,39.16,184.707990,7.662346,1950.963492,22.607795,76.680901,0,0,2,1
44110,101834,2013,M,82.0,Jujuy,38,SAN PEDRO,63,SAN PEDRO DE JUJUY,2013-01-02,...,39.16,184.707990,7.662346,1950.963492,22.607795,76.680901,0,0,2,1
52485,111660,2014,M,60.0,Salta,66,CAPITAL,28,SALTA,2014-01-02,...,41.91,174.493477,9.045500,1233.520144,28.420736,81.483862,0,0,3,1
52770,111979,2014,M,65.0,Salta,66,ORAN,126,SAN RAMON DE LA NUEVA ORAN,2014-01-02,...,57.87,174.493477,9.045500,1233.520144,28.420736,81.483862,0,0,3,1
77301,138880,2017,F,66.0,Jujuy,38,YAVI,112,LA QUIACA,2017-01-02,...,63.33,134.999975,5.834015,1632.969667,19.442558,64.962440,0,0,0,1
78209,139928,2017,M,16.0,Jujuy,38,GRAL MANUEL BELGRANO,21,SAN SALVADOR DE JUJUY,2017-01-02,...,44.96,134.999975,5.834015,1632.969667,19.442558,64.962440,0,0,0,1
